In [0]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_paths = [os.path.expanduser('~/data{}'.format(i)) for i in range(10)]
try:
  [os.makedirs(local_download_path) for local_download_path in local_download_paths]
except: pass

ids = ['1yGDDIp1VvuOrGmdRYA3ubrMC5thfgeg9', '1AG2h45GycGe60UaHDHqBN6zIVwcuQzix',
      '1VZxnT934IIcKFKTjQVXDOAPdeKbGgDRT', '1zLTbEVnz8IYo6PuwKrfuZl8kLRgYFQHu',
      '1KPjMbKb6jOJ3V6W6niay8O0yfApymDK8', '1e8dfhaIyUwFU8-RcDCf0MG5_9vqOKIxb',
      '1WHf_7EklerFYfLjPRHGbI4FVWForZuu9', '1rpyTIwmLUPRuOfatzhUjGwtnHpOa0TaO',
      '1DXe6LmSKA7UAxwFd_-xG7508chU-Ys1E', '1Vgjkw3ra8Q0QwlcLVXXio9dHlFp1nYSN']

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
#file_lists = [drive.ListFile({'q': "{} in parents".format('{}'.format(oneid))}).GetList() for oneid in ids]

file_lists = []
file_lists = [drive.ListFile({'q': "'{}' in parents".format(oneid)}).GetList() for oneid in ids]
for file_list,local_download_path in zip(file_lists,local_download_paths):  
  for f in file_list:
    # 3. Create & download by id.
    #print('title: %s, id: %s' % (f['title'], f['id']))
    fname = os.path.join(local_download_path, f['title'])
    #print('downloading to {}'.format(fname))
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)
  print("done with {}".format(local_download_path))


#with open(fname, 'r') as f:
#  print(f.read())

In [0]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_paths = [os.path.expanduser('~/data{}'.format(i)) for i in range(10)]

In [0]:
from matplotlib import pyplot
import numpy as np
from skimage import io
import glob
import random
import tensorflow as tf

In [197]:
os.path.expanduser('~/data')

'/content/data'

In [3]:
os.listdir('/content')

['datalab',
 '.local',
 '.cache',
 '.forever',
 '.rnd',
 '.config',
 '.ipython',
 '.keras']

In [199]:

np.asarray(io.imread('/content/data7/IMG_5874.JPG')).shape

(3024, 3024, 3)

In [200]:
#getting the path to every image. Is a nested list. List contains 10 lists. Each of the 10 lists
#contains about 200 image paths

img_names = [[os.path.join(local_download_path,img) for img in os.listdir(local_download_path)] for local_download_path in local_download_paths]

#Example: first five images of the digit 3

img_names[3][0:5]

['/content/data3/IMG_5850.JPG',
 '/content/data3/IMG_4499.JPG',
 '/content/data3/IMG_5799.JPG',
 '/content/data3/IMG_1141.JPG',
 '/content/data3/IMG_1315.JPG']

In [201]:
i = 0
for dig in img_names:
  for img,i in zip(dig,range(len(dig))):
    if(io.imread(img).shape != (100,100,3)):
      print(img)
      print(i)
#print (i)


/content/data7/IMG_5978.JPG
34
/content/data7/IMG_5874.JPG
46
/content/data7/IMG_5935.JPG
73


In [202]:
#img_ctrs keeps track of the images used from each digit set
#every time a batch is gotten, a random integer 'ind' is seleced b/w 0 and 9 (to select the digit)
#img_ctrs[ind] is the position of the image in 'ind' digit set that is added to the batch
#img_ctrs[ind] is then incremented

img_ctrs = [0 for i in range(10)]
max_imgs_per_dig = 170
my_batch_size = 50

#the 10 labels 
one_hot_vectors = [[(lambda i,j: int(i == j))(i,j)  for i in range(10)] for j in range(10)]
one_hot_vectors = np.array(one_hot_vectors)
one_hot_vectors

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [0]:
def get_batch_and_labels(batch_size):
  batch1 = []
  labels1 = []
  for i in range(batch_size):
    ind = random.randint(0,9)
    if(img_ctrs[ind] > max_imgs_per_dig):
      img_ctrs[ind] = 0
    image = io.imread(img_names[ind][img_ctrs[ind]]).astype(float)
    
    if(image.shape != (100,100,3)):
      img_ctrs[ind] += 1
      if(img_ctrs[ind] > max_imgs_per_dig):
        img_ctrs[ind] = 0
      image = io.imread(img_names[ind][img_ctrs[ind]]).astype(float)
    
    #print(img_names[ind][img_ctrs[ind]])
    assert(image.shape == (100,100,3))
    img_ctrs[ind] += 1
    batch1.append(image)
    labels1.append(one_hot_vectors[ind])
  return np.array(batch1),np.array(labels1)
    
def get_test_batch_and_labels():
  batch1 = []
  labels1 = []
  for dig,label in zip(img_names,one_hot_vectors):
    for i in range(171,201):
      image = (io.imread(dig[i])).astype(float)
      if(image.shape != (100,100,3)):
        continue
      batch1.append(image)
      labels1.append(label)
  return np.array(batch1),np.array(labels1)

In [0]:
#batch, labels = get_batch_and_labels(my_batch_size)
#type(batch)
#print(img_ctrs)
#print(labels.shape)

In [0]:
n_epochs = 2000
minibatch_size = 10
lr = 1e-4
keep = 0.5

In [0]:
x = tf.placeholder(tf.float32, shape = [None, 100, 100, 3])

y_ = tf.placeholder(tf.float32, shape = [None, 10])

In [0]:
def weight_variable(shape):
    """Initializes weights randomly from a normal distribution
    Params: shape: list of dimensionality of the tensor to be initialized
    """
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Initializes the bias term randomly from a normal distribution.
    Params: shape: list of dimensionality for the bias term.
    """
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    """Performs a convolution over a given patch x with some filter W.
    Uses a stride of length 1 and SAME padding (padded with zeros at the edges)
    Params:
    x: tensor: the image to be convolved over
    W: the kernel (tensor) with which to convolve.
    """
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# strides is a length-4 list that specifies the amount to move for each dimension of our input x. 
# the dimensions correspond to the following (in order): batch_size, length of image, width of image, # of channels in image

def max_pool_2x2(x):
    """Performs a max pooling operation over a 2 x 2 region"""
    # ksize: we only want to take the maximum over 1 example and 1 channel. 
    # the middle elements are 2 x 2 because we want to take maxima over 2 x 2 regions
    
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME') # stride 2 right and 2 down

In [0]:
W_conv1 = weight_variable([3, 3, 3, 32]) # 5 x 5 kernel, across an image with 3 channels to 32 channels
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [0]:
W_conv2 = weight_variable([3, 3, 32, 64]) # 5 x 5 kernel, across an "image" with 32 channels to 64 channels
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [210]:
W_conv3 = weight_variable([3, 3, 64, 128]) # 5 x 5 kernel, across an "image" with 32 channels to 64 channels
b_conv3 = bias_variable([128])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)
h_pool3

<tf.Tensor 'MaxPool_41:0' shape=(?, 13, 13, 128) dtype=float32>

In [0]:
W_conv4 = weight_variable([3, 3, 128, 256]) # 5 x 5 kernel, across an "image" with 32 channels to 64 channels
b_conv4 = bias_variable([256])
h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4)
#h_pool4 = max_pool_2x2(h_conv4)
#h_pool4

In [0]:
W_conv5 = weight_variable([3, 3, 256, 512]) # 5 x 5 kernel, across an "image" with 32 channels to 64 channels
b_conv5 = bias_variable([512])
h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5) + b_conv5)
#h_pool4 = max_pool_2x2(h_conv4)
#h_pool4

In [213]:
W_conv6 = weight_variable([3, 3, 512, 1024]) # 5 x 5 kernel, across an "image" with 32 channels to 64 channels
b_conv6 = bias_variable([1024])
h_conv6 = tf.nn.relu(conv2d(h_conv5, W_conv6) + b_conv6)
h_pool4 = max_pool_2x2(h_conv6)
h_pool4

<tf.Tensor 'MaxPool_42:0' shape=(?, 7, 7, 1024) dtype=float32>

In [0]:
W_fc1 = weight_variable([7 * 7 * 1024, 1024]) # This shape can be determined by plugging in something random * 64, and seing the resulting error. 
b_fc1 = bias_variable([1024])
h_pool4_flat = tf.reshape(h_pool4, [-1, 7 * 7 * 1024])
h_fc1 = tf.nn.relu(tf.matmul(h_pool4_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [0]:
W_fc2 = weight_variable([1024, 256])
b_fc2 = bias_variable([256])
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

In [0]:
W_fc3 = weight_variable([256, 10])
b_fc3 = bias_variable([10])
y_out = tf.matmul(h_fc2_drop, W_fc3) + b_fc3

In [0]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_out, labels = y_)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_, axis = 1), tf.argmax(y_out, axis = 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

In [0]:
with tf.Session() as sess:
  sess.run(init)
  for i in range(n_epochs):
    training_inputs, training_labels = get_batch_and_labels(minibatch_size)
    if i % 100 == 0:
      print("epoch:{}".format(i))
      #print(img_ctrs)
      #print(training_labels)
      train_acc = accuracy.eval(feed_dict = {x: training_inputs, y_: training_labels, keep_prob : 1.0})
      print("training accuracy: {}".format(train_acc))
    sess.run([train_step], feed_dict = {x: training_inputs, y_: training_labels, keep_prob : keep})
  test_inputs, test_labels = get_test_batch_and_labels();
  test_acc = accuracy.eval(feed_dict = {x: test_inputs, y_: test_labels, keep_prob : 1.0})
  print("test accuracy: {}".format(test_acc))

epoch:0
training accuracy: 0.10000000149011612
epoch:100
training accuracy: 0.10000000149011612
epoch:200
training accuracy: 0.20000000298023224
epoch:300
training accuracy: 0.0
epoch:400
training accuracy: 0.20000000298023224
epoch:500
training accuracy: 0.10000000149011612
epoch:600
training accuracy: 0.0
epoch:700
training accuracy: 0.10000000149011612
epoch:800
training accuracy: 0.30000001192092896
epoch:900
training accuracy: 0.30000001192092896
epoch:1000
training accuracy: 0.10000000149011612
epoch:1100
training accuracy: 0.10000000149011612
epoch:1200
training accuracy: 0.30000001192092896
epoch:1300
training accuracy: 0.20000000298023224
